In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import sys, os, socket
print(os.getcwd(), socket.gethostname())

out_dir = 'out/01c__multimer-v1__r3__msad512'

/mnt/home/ajasja/projects/2022-03-03__AF2_shorter_DHD jojo


In [2]:
import numpy
import numpy as np

from glob import glob
import os

import truncator
import truncator as u

import pymol
from pymol import cmd
from truncator.pymol_utils import *

In [3]:

pdbs=glob(f'{out_dir}/*.pdb')
print (f"Found pdbs: {len(pdbs)}")

Found pdbs: 720


In [4]:
def make_rosetta_command(xml_script, cmd_in, nstruct=1, target_res='', suffix='""', layer_type='sasa',
skip_design=False, design_type='fast_design', out_name=None, num=None, tee=False):
    """Makes a rosetta command line."""""
    import os
    cmd = ""

    #rosetta_bin = "/software/rosetta/latest/bin/rosetta_scripts"
    rosetta_bin = " /home/ajasja/Rosetta/main/source/bin/rosetta_scripts.linuxgccrelease"
    #rosetta_bin = "/home/bcov/ppi/tutorial_build/main/source/bin/rosetta_scripts"
    #rosetta_bin = "/software/rosetta/latest/bin/rosetta_scripts"
    #rosetta_bin = "/home/brunette/src/Rosetta_dev/main/source/bin/rosetta_scripts.hdf5.linuxgccrelease"
    #-holes:dalphaball !/home/bcov/dev_rosetta/main/source/external/DAlpahBall/DAlphaBall.gcc \
    #/software/rosetta/latest/bin/rosetta_scripts
    
    
    
    base_name = u.basename_noext(cmd_in)+suffix
    if not num is None:
        base_name = str(num)+"__"+base_name
    out_path = f"{out_name}/{base_name}"
    log_name =  f"{out_name}/{base_name}/{base_name}.log"
    
    if skip_design:
        do_fast_design_str = "<!--"
        do_repack_str = "<!--"
    else:
        if design_type=='fast_design':
            do_fast_design_str = '""'
            do_repack_str = "<!--"
        else:
            do_fast_design_str = "<!--"
            do_repack_str = '""'
        
            

    if layer_type=='sasa':
        do_sasa_str='""'
        do_neighbors_str = "<!--"
    else:
        do_sasa_str='<!--'
        do_neighbors_str = '""'
    cmd += f"mkdir -p {out_path} && "
    
    flags = f"""\
            -out::file::pdb_comments \
            -beta_nov16  \
            -s {cmd_in} \
            -in:file:native {cmd_in} \
            -parser:protocol {xml_script} \
            -parser:script_vars do_fast_design="{do_fast_design_str}"  \
            -parser:script_vars do_repack="{do_repack_str}"  \
            -parser:script_vars target_res="{target_res}"  \
            -parser:script_vars do_neighbors="{do_neighbors_str}"  \
            -parser:script_vars do_sasa="{do_sasa_str}"  \
            -suffix {suffix} \
            -nstruct {nstruct} \
            -overwrite 1 \
            -out:pdb 1 \
            -out:level 300 \
            -out::path::all {out_path} \
            -out:file:scorefile_format json \
            -renumber_pdb true \
            -include_current \
            -detect_disulf false \
            -ignore_unrecognized_res true \
            -mute protocols.rosetta_scripts.ParsedProtocol.REPORT \
            -out:levels core.select.residue_selector.SecondaryStructureSelector:warning \
            -out:levels core.select.residue_selector.PrimarySequenceNeighborhoodSelector:warning \
            -holes:dalphaball input/DAlphaBall.gcc \
            """.replace("            "," ")
    

    if tee:
        redir_str = " | tee "
    else:
        redir_str = " > "
    cmd += "{ \\time -v " + rosetta_bin + " " + flags +  " 2>&1 ;} " + redir_str + log_name
    return cmd


In [5]:
tasks_name = f"{out_dir}/02_calc_interface"
tasks_file_name = tasks_name+".tasks"


with open(tasks_file_name, "w+") as f:
    for pdb in pdbs:
        print(
           make_rosetta_command("02_calc_interface.xml", pdb, suffix="relaxed",
           layer_type='sasa', nstruct=1, out_name=f'{out_dir}/rosetta',  
           design_type='relax'),
        file=f)
    


cmds = u.read_file_lines(tasks_file_name)
#print(len(cmds))
#cmds
print(cmds[0]); print(len(cmds))


mkdir -p out/01c__multimer-v1__r3__msad512/rosetta/P7_P6_model_4_multimer_seed_0_unrelaxedrelaxed && { \time -v  /home/ajasja/Rosetta/main/source/bin/rosetta_scripts.linuxgccrelease  -out::file::pdb_comments  -beta_nov16   -s out/01c__multimer-v1__r3__msad512/P7_P6_model_4_multimer_seed_0_unrelaxed.pdb  -in:file:native out/01c__multimer-v1__r3__msad512/P7_P6_model_4_multimer_seed_0_unrelaxed.pdb  -parser:protocol 02_calc_interface.xml  -parser:script_vars do_fast_design="<!--"   -parser:script_vars do_repack=""""   -parser:script_vars target_res=""   -parser:script_vars do_neighbors="<!--"   -parser:script_vars do_sasa=""""   -suffix relaxed  -nstruct 1  -overwrite 1  -out:pdb 1  -out:level 300  -out::path::all out/01c__multimer-v1__r3__msad512/rosetta/P7_P6_model_4_multimer_seed_0_unrelaxedrelaxed  -out:file:scorefile_format json  -renumber_pdb true  -include_current  -detect_disulf false  -ignore_unrecognized_res true  -mute protocols.rosetta_scripts.ParsedProtocol.REPORT  -out:level

In [7]:
!{cmds[0]}


In [8]:
GROUP_SIZE=20
task_list=tasks_file_name
from math import ceil; num_tasks = ceil(len(cmds)/GROUP_SIZE)
!export GROUP_SIZE={GROUP_SIZE}; sbatch --mem=3G -p short -J {task_list} -o {task_list}.out -e {task_list}.err -a 1-{num_tasks} /home/ajasja/scripts/wrapper_digs_array_job_group.sh {task_list} 

Submitted batch job 24324545


In [9]:
!squeue -u $USER


ARRAY_JO ARRAY_TA PRIORI PARTITIO USER       START_TIME           ST TIME     MIN_ CPUS NAME                 NODELIST REASON               
24324545 1        1037   short    ajasja     2022-03-15T06:58:01  CF 0:01     3G   1    out/01c__multimer-v1 dig138   None                 
24324545 2        1037   short    ajasja     2022-03-15T06:58:01  CF 0:01     3G   1    out/01c__multimer-v1 dig145   None                 
24324545 3        1037   short    ajasja     2022-03-15T06:58:01  CF 0:01     3G   1    out/01c__multimer-v1 dig201   None                 
24324545 4        1037   short    ajasja     2022-03-15T06:58:01  CF 0:01     3G   1    out/01c__multimer-v1 dig201   None                 
24324545 5        1037   short    ajasja     2022-03-15T06:58:01  CF 0:01     3G   1    out/01c__multimer-v1 dig89    None                 
24324545 6        1037   short    ajasja     2022-03-15T06:58:01  CF 0:01     3G   1    out/01c__multimer-v1 dig89    None                 
24324545 7        10

In [ ]:
# run Z:/projects/truncator/truncator/pymol_utils.py
load_rosetta_pdb('mALb8_0001.pdb', per_res_metrics='per_res_sasa per_res_nc')
# run Z:/projects/truncator/truncator/pymol_utils.py
cbs per_res_sasa, range=-100 0 100, colors=green white red
cbs per_res_nc, range=-5 0 5, colors=green red white
cbs per_res_sap, range=-5 0 5, colors=green white red
cbs total, range=-5 0 5, colors=green white red

select core, p.per_res_sasa<19
select bdry, p.per_res_sasa>19 and p.per_res_sasa<40
select surf, p.per_res_sasa>40 

color white, core
color orange, bdry
color red, surf

color cyan
color tv_green, interface_by_vec
color marine, interface_near

In [ ]:
def load_score_file_to_dict(score_file):
    """Loads a Rosetta Json file to a dictionary indexed by decoy. In case of duplicate entries, the last one wins"""
    import json
    sc = {}
    with open(score_file) as f:
        for line in f:
            row = json.loads(line)
            #print(row)
            sc[row['decoy']] = row
    return sc

load_score_file_to_dict("out/01_sap_redesign_bcov/BW1/score.sc");